# Region Class Definition

## Setup

### Imports

In [1]:
from typing import List

# Shared Packages
import pandas as pd
from shapely.geometry import MultiPolygon, Polygon


# Local functions and classes
from types_and_classes import *
from utilities import *
from debug_tools import *
from structure_slice import *
from metrics import *
from relations import *

### Global Settings

In [2]:
PRECISION = 2

In [3]:
%matplotlib inline

## Need to do boundary checks by region not by structure
- Make a Region class with the following attributes:
    - ROI: ROI_Type, 
    - slice: SliceIndexType, 
    - is_hole: bool,
    - is_boundary: bool,
    - region_labels: List[str], 
    - polygon: ContourType
- Step through all structures and separate each slice into individual polygons 
    and holes (Regions)
- Apply region labels, based on overlapping polygons.
  - Set unique labels for each region on the first slice: 'a', 'b' ...
  - find overlapping polygons on the next slice and give them the same 
      region labels.
  - A polygon can be in multiple regions.
- If a polygon in the current region is not matched to a polygon in the next 
    region, it is a boundary polygon.
- If a polygon in the next region is not matched with a polygon in the current 
    region, it is a boundary polygon. Assign it to a new region
- Store as a list of dictionaries with keys: ROI, slice, is_hole, polygon
- convert to a DataFrame and sort on ROI and slice


In [4]:
class Region:
    def __init__(self, roi: ROI_Type, slice: SliceIndexType,
                 polygon: ContourType, is_hole: bool = False,
                 is_boundary: bool = False):
        self.roi = roi
        self.slice = slice
        self.is_hole = is_hole
        self.is_boundary = is_boundary
        self.region_labels = []
        self.polygon = polygon

    def __repr__(self):
        return ''.join([f'Region(roi={self.roi}, ',
                        f'slice={self.slice}, ',
                        f'is_hole={self.is_hole}, ',
                        f'is_boundary={self.is_boundary}, ',
                        f'region_labels={self.region_labels}, ',
                        #f'polygon={self.polygon})'
            ])

    def part_of(self, other: 'Region') -> bool:
        # Check if the region is part of another region
        # The combined three checks are done to ensure that If the region is a
        # hole, it is not part of the parent region.  This is done by excluding
        # the region boundaries and only testing the interiors.
        # An alternative approach would be to use the relate method and just
        # test the first bit.
        shared_interior = (self.polygon.overlaps(other.polygon) |
                        self.polygon.contains_properly(other.polygon) |
                        other.polygon.contains_properly(self.polygon))
        return shared_interior


In [5]:
def expand_region_table(regions_dict: dict[ROI_Type, dict[SliceIndexType, dict[str, Region]]]) -> pd.DataFrame:
    expanded_data = []
    for roi, slices in regions_dict.items():
        for slice_index, regions in slices.items():
            for region in regions:
                for label in region.region_labels:
                    expanded_data.append({
                        'ROI': roi,
                        'Slice': slice_index,
                        'Label': label,
                        'Region': region
                    })
    return expanded_data

In [6]:
# Function to create Region instances from slice-table DataFrame
def create_regions_from_slice_table(slice_table: pd.DataFrame) -> dict[ROI_Type, dict[SliceIndexType, list[Region]]]:
    regions_dict = {}
    for roi in slice_table.columns:
        regions_dict[roi] = {}
        previous_regions = []
        # Iterate over slices in the slice_table for a given ROI
        for slice_index, structure_slice in slice_table[roi].items():
            # Create a list of Region instances for each slice
            if slice_index not in regions_dict[roi]:
                regions_dict[roi][slice_index] = []
            # Create Region instances for each polygon and hole in the slice
            if not empty_structure(structure_slice):
                for polygon in structure_slice.contour.geoms:
                    # Create Region instances for each polygon in the slice
                    # Note: the polygon includes its holes.
                    region = Region(roi, slice_index, polygon, is_hole=False,
                                    is_boundary=False)
                    regions_dict[roi][slice_index].append(region)
                    # Create Region instances for each hole in the polygon
                    for interior in polygon.interiors:
                        hole = Polygon(interior)
                        region_hole = Region(roi, slice_index, hole,
                                             is_hole=True, is_boundary=False)
                        regions_dict[roi][slice_index].append(region_hole)
                # Set unique labels for each region on the first slice
                if slice_index == slice_table[roi].first_valid_index():
                    for idx, region in enumerate(regions_dict[roi][slice_index]):
                        region.region_labels.append(chr(97 + idx))  # 'a', 'b', 'c', ...
                else:
                    # Find overlapping polygons and give them the same region labels
                    for region in regions_dict[roi][slice_index]:
                        matched = False
                        for prev_region in previous_regions:
                            if region.part_of(prev_region):
                                region.region_labels.extend(prev_region.region_labels)
                                matched = True
                                break
                        if not matched:
                            region.is_boundary = True
            # Mark polygons in the previous region as boundary if not matched
            for prev_region in previous_regions:
                if not any(region.part_of(prev_region) for region in regions_dict[roi][slice_index]):
                    prev_region.is_boundary = True
            previous_regions = regions_dict[roi][slice_index]

    return regions_dict

In [7]:
def dual_embedded_cylinder():
    slice_spacing = 0.1
    # Body structure defines slices in use
    body = make_vertical_cylinder(roi_num=0, radius=12, length=1, offset_z=-0.5,
                                  spacing=slice_spacing)
    primary_cylinder = make_vertical_cylinder(roi_num=1, radius=5, length=0.7,
                                              offset_z=-0.3,
                                              spacing=slice_spacing)
    left_hole = make_vertical_cylinder(roi_num=1, radius=2, length=0.5,
                                       offset_x=-2.5, offset_z=-0.2,
                                       spacing=slice_spacing)
    right_hole = make_vertical_cylinder(roi_num=1, radius=2, length=0.5,
                                       offset_x=2.5, offset_z=-0.2,
                                       spacing=slice_spacing)
    # Two concentric cylinders different z offsets
    middle_cylinder = make_vertical_cylinder(roi_num=2, radius=1, length=0.5,
                                             offset_x=2.5, offset_z=-0.2,
                                             spacing=slice_spacing)

    # combine the contours
    slice_data = pd.concat([body, primary_cylinder, left_hole, right_hole, middle_cylinder])
    # convert contour slice data into a table of slices and structures
    slice_table = make_slice_table(slice_data, ignore_errors=True)
    return slice_table


In [8]:
slice_table = dual_embedded_cylinder()
regions_dict = create_regions_from_slice_table(slice_table)


In [11]:
pd.DataFrame(regions_dict)

,0,1,2
-0.5,"[Region(roi=0, slice=-0.5, is_hole=False, is_b...",[],[]
-0.4,"[Region(roi=0, slice=-0.4, is_hole=False, is_b...",[],[]
-0.3,"[Region(roi=0, slice=-0.3, is_hole=False, is_b...","[Region(roi=1, slice=-0.3, is_hole=False, is_b...",[]
-0.2,"[Region(roi=0, slice=-0.2, is_hole=False, is_b...","[Region(roi=1, slice=-0.2, is_hole=False, is_b...","[Region(roi=2, slice=-0.2, is_hole=False, is_b..."
-0.1,"[Region(roi=0, slice=-0.1, is_hole=False, is_b...","[Region(roi=1, slice=-0.1, is_hole=False, is_b...","[Region(roi=2, slice=-0.1, is_hole=False, is_b..."
0.0,"[Region(roi=0, slice=0.0, is_hole=False, is_bo...","[Region(roi=1, slice=0.0, is_hole=False, is_bo...","[Region(roi=2, slice=0.0, is_hole=False, is_bo..."
0.1,"[Region(roi=0, slice=0.1, is_hole=False, is_bo...","[Region(roi=1, slice=0.1, is_hole=False, is_bo...","[Region(roi=2, slice=0.1, is_hole=False, is_bo..."
0.2,"[Region(roi=0, slice=0.2, is_hole=False, is_bo...","[Region(roi=1, slice=0.2, is_hole=False, is_bo...","[Region(roi=2, slice=0.2, is_hole=False, is_bo..."
0.3,"[Region(roi=0, slice=0.3, is_hole=False, is_bo...","[Region(roi=1, slice=0.3, is_hole=False, is_bo...",[]
0.4,"[Region(roi=0, slice=0.4, is_hole=False, is_bo...",[],[]


In [10]:
expand_region_table(regions_dict)


[{'ROI': 0,
  'Slice': -0.5,
  'Label': 'a',
  'Region': Region(roi=0, slice=-0.5, is_hole=False, is_boundary=True, region_labels=['a'], },
 {'ROI': 1,
  'Slice': -0.3,
  'Label': 'a',
  'Region': Region(roi=1, slice=-0.3, is_hole=False, is_boundary=False, region_labels=['a'], },
 {'ROI': 1,
  'Slice': -0.2,
  'Label': 'a',
  'Region': Region(roi=1, slice=-0.2, is_hole=True, is_boundary=True, region_labels=['a'], },
 {'ROI': 1,
  'Slice': -0.2,
  'Label': 'a',
  'Region': Region(roi=1, slice=-0.2, is_hole=True, is_boundary=True, region_labels=['a'], },
 {'ROI': 2,
  'Slice': -0.2,
  'Label': 'a',
  'Region': Region(roi=2, slice=-0.2, is_hole=False, is_boundary=True, region_labels=['a'], }]